In [1]:
# import-
%run profile
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [12]:
# make squadgame function

def squadgame(csvfile):
    global squadgroup,squad,orgsquad,pubg,result
    global X,y,rg,X_train,y_train,X_test,y_test
    global killhack,wphack,DBNOshack,damagehack,assistshack,boostshack,roadkhack 
    pubg=pd.read_csv(csvfile)
    squad = pubg[pubg['matchType'].isin(['squad','squad-fpp'])]
    orgsquad=squad.copy()
    # 걷지 않고 무기를 사용하는 이상치 삭제
    walk0 = squad[squad.walkDistance==0]
    wphack=walk0[walk0.weaponsAcquired>0]
    
    # 걷지 않고 킬 하는 이상치 삭제
    killhack= walk0[walk0.kills>0]
    
    # 걷지않고 도핑하는 이상치 삭제
    boostshack = walk0[walk0.boosts>0]
    
    # 걷지않고 데미지 입히는 이상치 삭제
    damagehack = walk0[walk0.damageDealt>0]
    
    # 걷지않고 어시스트 이상치 삭제
    assistshack = walk0[walk0.assists>0]
    
    # 걷지않고 기절시키는 이상치 삭제
    DBNOshack = walk0[walk0.DBNOs>0]
    
    # 차를 탑승하지 않고 로드킬하는 이상치 삭제
    ride0=squad[squad.rideDistance==0]
    roadkhack=ride0[ride0.roadKills>0]
    
    result1=pd.merge(wphack,killhack,how='outer')
    result2=pd.merge(result1,boostshack,how='outer')
    result3=pd.merge(result2,damagehack,how='outer')
    result4=pd.merge(result3,assistshack,how='outer')
    result5=pd.merge(result4,DBNOshack,how='outer')
    result=pd.merge(result5,roadkhack,how='outer')
    
    squad=squad.drop(squad[squad.Id.isin(result.Id)].index)

    squadgroup=squad.drop(['Id','matchType','matchId'],axis=1).groupby(['groupId']).mean()
#     squad=squad[['Id','walkDistance','weaponsAcquired','heals','matchDuration','boosts','killPlace','winPlacePerc']].set_index('Id')
    print("총 {}명의 사용자 중 핵사용 의심 플레이어는 {}명 이며 전체의 {:.3f}%입니다\n"
          .format(len(orgsquad),len(result),len(squad)/len(orgsquad)*100))
    print("이상 인원을 제외 {}팀이 플레이하였습니다\n".format(len(squadgroup)))
    
    print("이상치 자료명은 'result'이며 정상 자료명은 'squad'입니다\n")
    
    print("핵 사용 의심 플레이어\n")
    print("이동 없이 무기소지 명단 : wphack ({}명)".format(len(wphack)))
    print("이동 없이 킬이 있는 명단 : killhack ({}명)".format(len(killhack)))
    print("이동 없이 아이템사용 명단 : boostshack ({}명)".format(len(boostshack)))
    print("이동 없이 데미지를 입힌 명단 : damagehack ({}명)".format(len(damagehack)))
    print("이동 없이 어시스트를 한 명단 : assistshack ({}명)".format(len(assistshack)))
    print("이동 없이 상대를 기절시킨 명단 : DBNOshack ({}명)".format(len(DBNOshack)))
    print("차량 탑승 없이 로드킬이 있는 명단 : roadkhack ({}명)\n".format(len(roadkhack)))
    
    print("중복인원 : {}명\n".format(len(wphack)+len(killhack)+
                              len(boostshack)+len(damagehack)+len(assistshack)+
                              len(DBNOshack)+len(roadkhack)-len(result)))
    
    X=squadgroup.drop(['winPlacePerc'],axis=1)
    y=squadgroup.winPlacePerc

    # RandomForestRegressor fitting & print score

    rg = RandomForestRegressor(n_jobs=-1,n_estimators=100,min_samples_leaf=1)
#     rg = RandomForestRegressor(n_jobs=-1)
    X_train,X_test,y_train,y_test = train_test_split(X,y)
    rg.fit(X_train,y_train)
    print("train data 예측률 : {:.3f}%".format(rg.score(X_train,y_train)*100))
    print("test data 예측률 : {:.3f}%".format(rg.score(X_test,y_test)*100))
    

In [3]:
squadgame("train_V2.csv")

총 2382712명의 사용자 중 핵사용 의심 플레이어는 4124명 이며 전체의 99.827%입니다

이상 인원을 제외 683000팀이 플레이하였습니다

이상치 자료명은 'result'이며 정상 자료명은 'squad'입니다

핵 사용 의심 플레이어

이동 없이 무기소지 명단 : wphack (3353명)
이동 없이 킬이 있는 명단 : killhack (35명)
이동 없이 아이템사용 명단 : boostshack (0명)
이동 없이 데미지를 입힌 명단 : damagehack (1121명)
이동 없이 어시스트를 한 명단 : assistshack (82명)
이동 없이 상대를 기절시킨 명단 : DBNOshack (142명)
차량 탑승 없이 로드킬이 있는 명단 : roadkhack (150명)

중복인원 : 759명

train data 예측률 : 98.951%
test data 예측률 : 92.553%


In [6]:
from sklearn.preprocessing import PolynomialFeatures

In [7]:
# feature_importances_ values chart

f_importances=DataFrame(rg.feature_importances_)
f_importances.index=list(X_train.columns)
f_importances.sort_values(by=0,ascending=False)

,0
walkDistance,0.798422
killPlace,0.085689
matchDuration,0.019872
boosts,0.011475
killStreaks,0.011046
kills,0.009920
rideDistance,0.008077
longestKill,0.007226
damageDealt,0.006645
numGroups,0.005768


In [8]:
# polynomial

poly=PolynomialFeatures(degree=2).fit(X_train)
poly_col=poly.get_feature_names(X_train.columns)
x_data_poly=DataFrame(poly.transform(X),columns=poly_col)


In [9]:
# poly CV

scores=cross_validate(rg,x_data_poly,y, cv=4,scoring=('r2','neg_mean_squared_error'),
                     return_train_score=True)
print(scores['test_neg_mean_squared_error'])
print(scores['train_r2'])
print(scores['test_r2'])

[-0.00673426 -0.00672524 -0.0067405  -0.00669563]
[0.98967464 0.98967711 0.9896669  0.98964246]
[0.926431   0.92614595 0.92634131 0.92660819]


In [11]:
# poly fitting & feature importance

rgpoly=RandomForestRegressor()
Xp_train,Xp_test,yp_train,yp_test = train_test_split(x_data_poly,y)
rgpoly.fit(Xp_train,yp_train)

f_importances=DataFrame(rgpoly.feature_importances_)
f_importances.index=list(Xp_train.columns)
f_importances.sort_values(by=0,ascending=False)


,0
numGroups walkDistance,6.616545e-01
maxPlace walkDistance,7.444611e-02
boosts walkDistance,7.378806e-02
killPlace matchDuration,6.604580e-02
killPlace killStreaks,1.189931e-02
killPlace^2,6.912137e-03
killPlace,6.789390e-03
killPlace kills,6.475829e-03
walkDistance weaponsAcquired,3.254045e-03
walkDistance^2,2.917505e-03
